# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-121947")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-121947
Azure region: southcentralus
Subscription id: a7e82b92-bbed-4497-a8df-009b71b94f9c
Resource group: aml-quickstarts-121947


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
amlcompute_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2',
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.dataset import Dataset
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import argparse
import os
import numpy as np
from sklearn.metrics import mean_squared_error
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from azureml.core.run import Run
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.dataset import Dataset

## Data Preparation

In [5]:

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
datasetUrl = "https://www.unsw.adfa.edu.au/unsw-canberra-cyber/cybersecurity/ADFA-NB15-Datasets/a%20part%20of%20training%20and%20testing%20set/UNSW_NB15_training-set.csv"

ds = Dataset.Tabular.from_delimited_files(datasetUrl)


In [7]:
def clean_data(data):

    x_df = data.to_pandas_dataframe().dropna()

    x_df = x_df.drop(['attack_cat'] , axis=1)
    x_df.service = LabelEncoder().fit_transform(x_df.service)
    x_df.state = LabelEncoder().fit_transform(x_df.state)
    x_df.proto = x_df.proto.replace(x_df.proto.value_counts())
    y_df = x_df.pop("label")
    
    return x_df , y_df

In [9]:
#from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

data = pd.concat([x,y] , axis=1)

trainData , testData =  train_test_split( data, test_size=0.3, random_state=123)

In [10]:
trainData.head()

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,label
84472,84473,1.209230,79946,0,2,10,8,2676,354,14.058534,...,1,1,1,0,0,0,1,2,0,0
148889,148890,0.000009,63283,2,3,2,0,114,0,111111.107200,...,15,15,21,0,0,0,15,21,0,1
84077,84078,0.000011,1150,0,3,2,0,1336,0,90909.090200,...,1,1,1,0,0,0,1,1,0,1
187,188,0.000011,63283,0,3,2,0,1154,0,90909.090200,...,5,1,63,0,0,0,5,62,0,0
38070,38071,0.001055,63283,2,0,2,2,146,178,2843.601894,...,2,1,2,0,0,0,6,2,0,0


## AutoMl Configuration and Experiment Run

In [11]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=15,
    task= 'classification',
    primary_metric= "AUC_weighted",
    
    training_data= trainData,
    label_column_name= 'label',
    n_cross_validations= 2)

In [12]:
remote_run = exp.submit(automl_config, show_output = True)

Running on local machine
Parent Run ID: AutoML_194469e3-2784-43c4-b372-fa01c9b00088

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing f

In [13]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [14]:

best_run, fitted_model = remote_run.get_output()

print(best_run)
print(fitted_model)

Run(Experiment: udacity-project,
Id: AutoML_194469e3-2784-43c4-b372-fa01c9b00088_0,
Type: None,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('MaxAbsScaler', MaxAbsScaler(copy...
                 LightGBMClassifier(boosting_type='gbdt', class_weight=None,
                                    colsample_bytree=1.0,
                                    importance_type='split', learning_rate=0.1,
                                    max_depth=-1, min_child_s

## Save and Register the best model from AutoML

In [48]:
joblib.dump(fitted_model, "model.pkl")

['model.pkl']

In [49]:
from azureml.core.model import Model
model = Model.register(model_path = "model.pkl",
                       model_name = "classification_model",
                       
                       description = "fraudlent detection",
                       workspace = ws)


Registering model classification_model


In [76]:
model

Model(workspace=Workspace.create(name='quick-starts-ws-121947', subscription_id='a7e82b92-bbed-4497-a8df-009b71b94f9c', resource_group='aml-quickstarts-121947'), name=classification_model, id=classification_model:3, version=3, tags={}, properties={})

## Update Score.py file

In [77]:

%%writefile score.py
import json
import numpy as np
import os
import pickle
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression

from azureml.core.model import Model

def init():
    global model
    # retrieve the path to the model file using the model name
    model_path = Model.get_model_path('classification_model')
    model = joblib.load(model_path)

def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    y_hat = model.predict(data)
    return json.dumps(y_hat.tolist())

Overwriting score.py


## Setup inference Config and Web Service Configuration

In [85]:
service_name = 'my-custom-env-service'
env = Environment.get(workspace=ws, name="AzureML-Tutorial")
env.python.conda_dependencies.add_pip_package("scikit-learn")
inference_config = InferenceConfig(entry_script='score.py', environment=env) 
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=4, enable_app_insights=True)


## Deploy the Model and Check the Scoring URI

In [88]:
service = model.deploy(workspace= ws , name = service_name , models =[model],inference_config= inference_config,
                      deployment_config=  aci_config)
service.wait_for_deployment(show_output= True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running............................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [89]:
print(service.scoring_uri)
print(service.swagger_uri)

http://0a269042-9938-4b32-97af-0b543813b94f.westus.azurecontainer.io/score
http://0a269042-9938-4b32-97af-0b543813b94f.westus.azurecontainer.io/swagger.json


In [100]:
test_new = testData.drop(['label'] ,axis =1)

In [102]:
test_new.head(1).values.tolist()

[[33884.0,
  0.32616,
  79946.0,
  0.0,
  2.0,
  28.0,
  28.0,
  4480.0,
  2976.0,
  168.628887,
  31.0,
  29.0,
  105960.2578,
  70394.89844,
  7.0,
  7.0,
  12.06863,
  12.059852,
  716.385152,
  19.053781,
  255.0,
  1883252194.0,
  4031733585.0,
  255.0,
  0.000826,
  0.00054,
  0.000286,
  160.0,
  106.0,
  0.0,
  0.0,
  5.0,
  0.0,
  3.0,
  1.0,
  1.0,
  2.0,
  0.0,
  0.0,
  0.0,
  2.0,
  4.0,
  0.0]]

## Consume the model 

In [107]:
import requests
import json

# URL for the web service
scoring_uri = 'http://0a269042-9938-4b32-97af-0b543813b94f.westus.azurecontainer.io/score'
# If the service is authenticated, set the key or token
#key = '<your key or token>'

# Two sets of data to score, so we get two results back
data = {"data":
            [[33884.0, 0.32616, 79946.0, 0.0, 2.0, 28.0, 28.0, 4480.0, 2976.0, 168.628887, 31.0, 29.0, 105960.2578, 70394.89844, 7.0, 7.0, 12.06863, 12.059852, 716.385152, 19.053781, 255.0, 1883252194.0, 4031733585.0, 255.0, 0.000826, 0.00054, 0.000286, 160.0, 106.0, 0.0, 0.0, 5.0, 0.0, 3.0, 1.0, 1.0, 2.0, 0.0, 0.0, 0.0, 2.0, 4.0, 0.0]]
        }
# Convert to JSON string
input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
#headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.text)

[1]


## Delete Service

In [108]:
service.delete()